In [2]:
# import all packages needed
import string 
import numpy as np
import pandas as pd
from matplotlib import pyplot
from base64 import b64decode as decode
import math
import torch
import torch.nn as nn 
import torch.fft as fft
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Data Processing / Cleaning

In [2]:
# use class base64 to decode waveform data
def to_array(wf):
    barr = bytearray(decode(wf))
    vals = np.array(barr)
    return vals.view(np.int16)

# read in data
exam_data = pd.read_csv("data/d_exam.csv").drop(columns = ["site_num", "patient_id_edit"])
waveform_data = pd.read_csv("data/d_waveform.csv")
lead_data = pd.read_csv("data/d_lead_data.csv").drop(columns = ["exam_id"])
diagnosis_data = pd.read_csv("data/d_diagnosis.csv").drop(columns = ["user_input"])

# add decoded data as a column to lead dataz
waveforms = list(lead_data['waveform_data'])
lead_data['decoded_waveform'] = [to_array(i) for i in waveforms]

# merge waveform data and lead data
waveform_lead = lead_data.merge(waveform_data, how = "left", left_on = "waveform_id", right_on = "waveform_id", suffixes = (None, None))

#  sort by exam id and lead id
waveform_lead.sort_values(by = ["waveform_id", "lead_id"], inplace = True)

waveform_lead.loc[:, ['exam_id', 'lead_id', 'decoded_waveform', 'waveform_type']]


# adding the diagnosis and labels
waveform_and_diag = pd.merge(waveform_lead[['exam_id', 'lead_id', 'decoded_waveform', 'waveform_type']], diagnosis_data[["exam_id", "Full_text", "Original_Diag"]], left_on= "exam_id", right_on="exam_id")


In [3]:
# concatenate all leads into a single array
waveform_lead_concat = waveform_lead.groupby(["exam_id", "waveform_type"])['decoded_waveform'].apply(lambda x: tuple(x)).reset_index()

# remove irregular observations, concat tuple into numpy array
waveform_lead_concat = waveform_lead_concat.drop([12,17], axis = 0)
waveform_lead_concat['decoded_waveform'] = waveform_lead_concat['decoded_waveform'].apply(lambda x: MinMaxScaler().fit_transform(np.vstack(x)))
waveform_lead_rhythm = waveform_lead_concat[waveform_lead_concat['waveform_type'] == "Rhythm"]
waveform_lead_median = waveform_lead_concat[waveform_lead_concat['waveform_type'] == "Median"]

waveform_lead_rhythm['decoded_waveform'][1].shape

(8, 2500)

In [4]:
# Adding the labels/sentences
exams = diagnosis_data["exam_id"].unique()

# Let's look over this tomorrow
diagnosis_data = diagnosis_data[diagnosis_data['Original_Diag'] == 1].dropna()
searchfor = ['previous', 'unconfirmed', 'compared', 'interpretation', 'significant']
diagnosis_data = diagnosis_data.loc[diagnosis_data['Full_text'].str.contains('|'.join(searchfor)) != 1]
#

diagnosis_data.sort_values(by=["exam_id", "statement_order"], inplace=True)
diagnoses = []
curr_id = 0
curr_string = ""
for i, row in diagnosis_data.iterrows():
    if row["statement_order"] == 1 and curr_string != "":
        curr_string = curr_string.lower().translate(str.maketrans('', '', string.punctuation))
        val = [curr_id, curr_string[1:]]
        diagnoses.append(val)
        curr_string = ""
        curr_id = row["exam_id"]

    if curr_id == 0:
        curr_id = row["exam_id"]
    
    curr_string += " " + row["Full_text"]

diagnosis_df = pd.DataFrame(diagnoses, columns = ['exam_id', 'diagnosis'])
waveform_lead_rhythm_diag = pd.merge(left=waveform_lead_rhythm, right=diagnosis_df, left_on='exam_id', right_on='exam_id')

#waveform_lead_rhythm_diag
waveform_lead_rhythm_diag

,exam_id,waveform_type,decoded_waveform,diagnosis
0,548759,Rhythm,"[[0.42857142857142855, 0.45454545454545453, 0....",normal sinus rhythm low voltage qrs borderline...
1,549871,Rhythm,"[[0.75, 0.7391304347826086, 0.7272727272727272...",sinus bradycardia otherwise normal ecg
2,550602,Rhythm,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",sinus tachycardia otherwise normal ecg
3,551485,Rhythm,"[[0.5, 0.5079365079365079, 0.5161290322580645,...",normal sinus rhythm normal ecg
4,552077,Rhythm,"[[0.5333333333333332, 0.53125, 0.5714285714285...",normal sinus rhythm normal ecg
5,552856,Rhythm,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",normal sinus rhythm with sinus arrhythmia mini...
6,553115,Rhythm,"[[0.0, 0.2, 0.375, 0.375, 0.375, 0.1875, 0.0, ...",atrial fibrillation abnormal ecg normal sinus ...


In [5]:
unique_words = set()
for num, sentence in diagnoses:
    for word in sentence.split():
        unique_words.add(word)
print(unique_words)

{'fibrillation', 'with', 'tachycardia', 'be', 'may', 'criteria', 'rhythm', 'borderline', 'arrhythmia', 'qrs', 'bradycardia', 'minimal', 'low', 'otherwise', 't', 'consider', 'ischemia', 'for', 'voltage', 'atrial', 'normal', 'sinus', 'abnormality', 'variant', 'inferior', 'abnormal', 'lvh', 'ecg', 'wave'}


## Encoder 1: ResNet Encoder

In [7]:
# HYPERPARAMETERS
J = 10 # max number of filters per class
LR = 1e-3

# define global max pooling
class global_max_pooling_1d(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        x, _ = torch.max(x, dim = 2)
        return(x)

# define resblock for neural nets
class ResBlock1D(nn.Module):
    def __init__(self, num_filters, kernel_size, padding, groups = 1, stride = 1):
        super(ResBlock1D, self).__init__()
        self.act = nn.ReLU()
        self.conv1d_1 = nn.Conv1d(num_filters, num_filters, kernel_size = kernel_size, padding = padding, groups = groups, stride = 1)
        self.conv1d_2 = nn.Conv1d(num_filters, num_filters, kernel_size = kernel_size, padding = padding, groups = groups, stride = 1)
        self.batch_norm_1 = nn.BatchNorm1d(num_filters)
        self.batch_norm_2 = nn.BatchNorm1d(num_filters)

    def forward(self, x):
        res = x
        x = self.batch_norm_1(self.act(self.conv1d_1(x)))
        x = self.batch_norm_2(self.act(self.conv1d_2(x)))
        return x + res

# build resent model and display the shape of feed through
conv_model = nn.Sequential()
init_channels = 8
for i in range(5):
    next_channels = 2 * init_channels
    conv_model.add_module('conv_{num}'.format(num = i), nn.Conv1d(in_channels = init_channels, out_channels = next_channels, kernel_size = 249, padding = 124, stride = 1))
    conv_model.add_module('act_{num}'.format(num = i), nn.ReLU())
    conv_model.add_module('batch_norm_{num}'.format(num = i), nn.BatchNorm1d(next_channels))
    conv_model.add_module('res_{num}'.format(num = i), ResBlock1D(num_filters = next_channels, kernel_size = 249, padding = 124))
    conv_model.add_module('act_res_{num}'.format(num = i), nn.ReLU())
    init_channels = next_channels
    
conv_model.add_module('conv_fin', nn.Conv1d(in_channels = init_channels, out_channels = 8, kernel_size = 249, padding = 124))
conv_model.add_module('act_fin', nn.ReLU())
conv_model.add_module('batch_fin', nn.BatchNorm1d(8))
print(conv_model)
print(conv_model(train_x).shape)

Sequential(
  (conv_0): Conv1d(8, 16, kernel_size=(249,), stride=(1,), padding=(124,))
  (act_0): ReLU()
  (batch_norm_0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res_0): ResBlock1D(
    (act): ReLU()
    (conv1d_1): Conv1d(16, 16, kernel_size=(249,), stride=(1,), padding=(124,))
    (conv1d_2): Conv1d(16, 16, kernel_size=(249,), stride=(1,), padding=(124,))
    (batch_norm_1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm_2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (act_res_0): ReLU()
  (conv_1): Conv1d(16, 32, kernel_size=(249,), stride=(1,), padding=(124,))
  (act_1): ReLU()
  (batch_norm_1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res_1): ResBlock1D(
    (act): ReLU()
    (conv1d_1): Conv1d(32, 32, kernel_size=(249,), stride=(1,), padding=(124,))
    (conv1d_2): Conv1d(32, 32, kernel_size=(249,), stride

## Encoder 2 - LSTM Encoder

In [17]:
# define hyperparameters 
hidden_layers = 250
embedding_dim = 8
num_words = len(dict_words)

class LSTM_EncoderDecoder(nn.Module):
    def __init__(self, h_dim, e_dim, word_list_length):
        super(ECG_LSTM, self).__init__()
        self.lstm = nn.LSTM(e_dim, h_dim, num_layers = 4, bidirectional = True)
        
    def forward(self, seq):
        seq_embedded = seq.view(len(seq), -1, embedding_dim)
        final_hidd, _ = self.lstm(seq_embedded)
        dec_seq = self.linear(final_hidd)
        return F.log_softmax(dec_seq, dim = 1)
    

## Encoder 3 - Basic Transformer Architecture with Multi-Head Attention

## Encoder 4 - FNET Transformer Architecture

In [3]:
class FeedForwardNet(nn.Module):
    def __init__(self, features, expansion, dropout):
        super(FeedForwardNet, self).__init__()
        self.linear_1 = nn.Linear(features, features * expansion)
        self.linear_2 = nn.Linear(features * expansion, features)
        self.dropout_1 = nn.Dropout(dropout)
        #self.dropout_2 = nn.Dropout(dropout)
        self.norm_1 = nn.LayerNorm(features)

    def forward(self, x):
        res = x
        x = F.relu(self.linear_1(x))
        x = self.dropout_1(x)
        x = self.linear_2(x)
        x = self.norm_1(x + res)
        return x
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
    
class FNETLayer(nn.Module):
    def __init__(self, features, expansion, dropout):
        super(FNETLayer, self).__init__()
        self.feed_forward = FeedForwardNet(features, expansion, dropout)
        self.norm_1 = nn.LayerNorm(features)
    
    def forward(self, x):
        res = x
        x = fft.fftn(x, dim = (-2, -1)).real
        x = self.norm_1(x + res)
        x = self.feed_forward(x)
        return x
    
class FNETEncoder(nn.TransformerEncoder):
    def __init__(self, features, expansion=2, dropout=0.5, num_layers=6):
        encoder_layer = FNETLayer(features, expansion, dropout)
        super().__init__(encoder_layer=encoder_layer, num_layers=num_layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
 
class Transpose(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x.transpose(1, 2)


## Decoder 1 - Huggingface GPT2 Decoder

In [8]:
# define tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2', config = GPT2Config(add_cross_attention = True))

# preprocess training labels and tokenize
train_labels = list(waveform_lead_rhythm_diag['diagnosis'])
inputs = tokenizer(train_labels, padding = True, verbose = False, return_tensors="pt")



Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.11.crossattention.bias', 'h.9.crossattention.c_proj.bias', 'h.7.crossattention.bias', 'h.5.crossattention.q_attn.weight', 'h.10.crossattention.c_proj.weight', 'h.0.crossattention.c_attn.weight', 'h.11.crossattention.q_attn.weight', 'h.5.crossattention.bias', 'h.8.crossattention.c_attn.weight', 'h.3.crossattention.bias', 'h.3.ln_cross_attn.weight', 'h.3.crossattention.q_attn.weight', 'h.11.ln_cross_attn.weight', 'h.3.crossattention.c_proj.bias', 'h.6.crossattention.masked_bias', 'h.10.crossattention.bias', 'h.8.crossattention.bias', 'h.7.crossattention.q_attn.weight', 'h.7.crossattention.masked_bias', 'h.6.crossattention.c_attn.weight', 'h.1.crossattention.c_attn.weight', 'h.4.ln_cross_attn.weight', 'h.11.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.10.crossattention.q_attn.weight', 'h.10.ln_cross_attn.weight', 'h.1.crossattention.q_attn.weight', '

In [ ]:
# pretrain decoder
torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
torch.autograd.set_detect_anomaly(True)

# set number of epochs
epochs = 100

for i in range(epochs):
    optimizer.zero_grad()
    outputs = model(**inputs, labels = inputs["input_ids"])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    
    print(loss)
    
torch.save(model.state_dict(), 'model/gpt2.pt')


## EncoderDecoder - FNET Encoder Huggingface Decoder

In [9]:
# create encoder decoder model with GPT2 
class CustEncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, embedder):
        super(CustEncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.pos_enb = PositionalEncoding(d_model = 768)
        self.embedder = embedder
    
    def forward(self, x):
        ecgs, labels = x
        x = self.embedder(ecgs).permute(2, 0, 1)
        x = self.pos_enb(x).permute(1, 0, 2)
        x = self.encoder(x)
        out = self.decoder(**labels, labels = labels["input_ids"], encoder_hidden_states = x.contiguous())
        return out
    
    def return_enc(self):
        return self.encoder

# define component models
conv_embedder = nn.Sequential(nn.Conv1d(in_channels = 8, out_channels = 350, kernel_size = 15, padding = 7, stride = 1),
                              nn.ReLU(),
                              nn.BatchNorm1d(350),
                              nn.Conv1d(in_channels = 350, out_channels = 768, kernel_size = 15, padding = 7, stride = 1),
                              nn.ReLU())
model.load_state_dict(torch.load('model/gpt2.pt'))
encoder = FNETEncoder(768, expansion = 2, dropout=0.1, num_layers = 6)
enc_dec_model = CustEncoderDecoder(encoder, model, conv_embedder)

# define all x, feed through to see if all is good
full_x = torch.tensor(list(waveform_lead_rhythm_diag['decoded_waveform'])).float()
#enc_dec_model((full_x, inputs))

tensor([[ 2.0693e-02,  1.1923e-02,  1.4417e-02,  ...,  2.1929e-02,
          6.0698e-03,  8.2075e-03],
        [ 3.7771e-03, -2.0714e-02, -1.9049e-02,  ..., -2.6400e-02,
         -2.4504e-02, -2.6173e-02],
        [-1.2160e-02, -2.7111e-03, -1.4916e-02,  ...,  3.0961e-02,
          1.1428e-02, -1.0093e-02],
        ...,
        [ 2.5049e-02,  1.5094e-02, -8.7690e-05,  ..., -1.8790e-02,
          6.0749e-03,  4.9255e-03],
        [ 9.5033e-03, -5.9113e-03,  8.5214e-03,  ...,  1.8400e-02,
          2.2191e-02,  1.4525e-02],
        [-2.4371e-02, -1.4399e-02,  1.1266e-02,  ..., -3.3196e-02,
          2.4634e-02,  3.1565e-02]])
tensor([ 0.0109,  0.0138,  0.0323,  ..., -0.0320,  0.0293, -0.0003])
tensor([[ 0.0063,  0.0158, -0.0018,  ...,  0.0103, -0.0120, -0.0195],
        [-0.0200, -0.0059, -0.0214,  ...,  0.0095,  0.0212, -0.0205],
        [ 0.0209,  0.0049,  0.0223,  ...,  0.0055, -0.0178, -0.0067],
        ...,
        [ 0.0177, -0.0179, -0.0160,  ..., -0.0021, -0.0052, -0.0084],
      

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [13]:
# train encoder decoder model!
torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(enc_dec_model.parameters(), lr = 1e-5)
torch.autograd.set_detect_anomaly(True)

# set number of epochs
epochs = 100

for i in range(epochs):
    optimizer.zero_grad()
    outputs = enc_dec_model((full_x, inputs))
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    
    print(loss)
    
torch.save(model.state_dict(), 'model/gpt2_enc_dec.pt')


tensor(0.0899, grad_fn=<NllLossBackward>)
tensor(0.0763, grad_fn=<NllLossBackward>)
tensor(0.0657, grad_fn=<NllLossBackward>)
tensor(0.0596, grad_fn=<NllLossBackward>)
tensor(0.0557, grad_fn=<NllLossBackward>)
tensor(0.0522, grad_fn=<NllLossBackward>)
tensor(0.0503, grad_fn=<NllLossBackward>)
tensor(0.0481, grad_fn=<NllLossBackward>)
tensor(0.0468, grad_fn=<NllLossBackward>)
tensor(0.0459, grad_fn=<NllLossBackward>)
tensor(0.0447, grad_fn=<NllLossBackward>)
tensor(0.0439, grad_fn=<NllLossBackward>)
tensor(0.0428, grad_fn=<NllLossBackward>)
tensor(0.0419, grad_fn=<NllLossBackward>)
tensor(0.0416, grad_fn=<NllLossBackward>)
tensor(0.0407, grad_fn=<NllLossBackward>)
tensor(0.0407, grad_fn=<NllLossBackward>)


KeyboardInterrupt: 